# End-to-End Memory Network

The End-to-End Memory network built with the memory_model.py script is designed to recieve a story and a question (query) as inputs and return a yes/no answer to the query.

### Load In Model

The first step is to load in the model structure and weights. The model is stored as a json object so we will use the model_from_json method from keras.models to load it in.

In [1]:
from keras.models import model_from_json
# load json and create model
with open('model.json', 'r') as f:
    model = model_from_json(f.read())
    
# load weights into model
model.load_weights("chatbot_150epochs.h5")
print("Loaded model from disk")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


### Load Vocabulary, Tokenizer, and Supporting Files

Next, we need to load the vocabulary that the model was trained on, as well as the tokenizer object that is used to vectorize the words in the stories, questions, and answers.

In [2]:
import ast
import pickle
with open('vocabulary.txt','r') as f:
   vocab = ast.literal_eval(f.read())
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open('parameters.txt','r') as f:
   params = ast.literal_eval(f.read())

In [3]:
# Words in the Vocabulary
print(vocab)

{'went', 'kitchen', 'discarded', 'milk', 'picked', 'left', 'Is', 'got', 'office', 'hallway', 'there', 'bedroom', 'travelled', 'moved', 'to', 'garden', 'down', 'Daniel', 'the', 'up', 'bathroom', '.', 'no', 'took', 'grabbed', 'journeyed', '?', 'dropped', 'in', 'put', 'John', 'back', 'football', 'Sandra', 'Mary', 'yes', 'apple'}


### Evaluate Custom Story/Question

We now have all the tools we need to test our model on a custom story and question. Keep in mind, we can only use words from the vocabulary that the model was trained on. (We also have to be mindful of the extra space between the last word in a sentence and the ending punctuation.)

In [9]:
# Define story variables
my_story = "Sandra travelled up to the bedroom . Daniel went in the office . Mary grabbed the milk ."
my_question = "Is Daniel in the office ?"
my_answer = 'yes'

We need to use the .split() method on our variables in order to store them in the same format as the training data. We then pass our data through our custom vectorize_stories function that we have defined in the network_functions.py script.

In [10]:
mydata = [(my_story.split(),my_question.split(),my_answer)]

In [11]:
from network_functions import vectorize_stories
my_story,my_ques,my_ans = vectorize_stories(mydata,
                                            word_index=tokenizer.word_index,
                                            max_story_len=params['max_story_len'],
                                            max_question_len=params['max_question_len'])

### Predict Answer

In [12]:
pred_results = model.predict(([ my_story, my_ques]))

In [15]:
import numpy as np

# Generate prediction from model
def prediction(pred_results):
    val_max = np.argmax(pred_results[0])
    for key, val in tokenizer.word_index.items():
        if val == val_max:
            k = key
    print("Predicted answer is: ", k)
    print("Probability of certainty was: ", str(round(100*pred_results[0][val_max],2)),"%")
    return {'prediction':k,'probability':pred_results[0][val_max]}

pred_ansnwer = prediction(pred_results)

Predicted answer is:  yes
Probability of certainty was:  88.13 %
